<a href="https://colab.research.google.com/github/prudhvichiru/FMML/blob/main/copy_of_module_01_lab_02_mlpractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1)Increasing the Percentage of Validation Set:

Pros: A larger validation set can provide a more robust estimate of your model's performance. It gives the model more data to generalize from, which can lead to a better assessment of its true performance.
Cons: With a larger validation set, you have fewer data points for training, which might result in slower convergence and potentially underfitting if your training set is too small.

Reducing the Percentage of Validation Set:

Pros: More data available for training can help your model learn better and potentially achieve higher accuracy on the training data.
Cons: A smaller validation set may lead to overfitting, where the model becomes too tailored to the training data and doesn't generalize well to unseen data. It may also provide a less reliable estimate of your model's true performance because it's based on fewer data points

2)Increasing Training Size:

A larger training set typically leads to better model generalization because the model has more data to learn from.
With a substantial training set, the validation set's performance becomes a more reliable predictor of test set accuracy.
As training size increases, the validation set accuracy tends to converge towards the test set accuracy.

Decreasing Training Size:

Smaller training sets may result in models that overfit (perform well on training data but poorly on unseen data).
The validation set's accuracy may not accurately estimate test set performance when training data is limited.

Hypothetical Graph:



X-axis: Training Set Size (Increasing from left to right)
Y-axis: Accuracy on Validation Set
In the graph, as the training set size increases, the accuracy on the validation set generally improves, reaching a plateau where further increases have diminishing returns. With smaller training set sizes, there's more variability in validation set accuracy, and it might not reach high levels

3)Training Data Size: By dedicating 70% to 80% of your data for training, you provide your model with enough information to learn the underlying patterns in your dataset. A larger training set often leads to better model generalization.

Validation Data Size: With 20% to 30% of your data in the validation set, you have a sufficiently sized sample to evaluate your model's performance reliably. This ensures that your validation accuracy or other metrics are indicative of how well your model generalizes to unseen data.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1)Yes, averaging the validation accuracy (or other performance metrics) across multiple splits, such as in k-fold cross-validation, can provide more consistent and robust results compared to a single data split. Here's why:

Reduced Variance: With a single data split, the performance of your model may be heavily influenced by the particular data points in that split. If you have a random or uneven distribution of data, this can lead to variability in the results. Averaging over multiple splits helps reduce this variance.

Better Generalization Estimate: Cross-validation ensures that your model is evaluated on different subsets of the data, which provides a more comprehensive assessment of its generalization performance. It can help identify whether your model's performance is consistent across various parts of the dataset or if it's sensitive to specific data points

2)Averaging validation accuracy across multiple splits, such as in k-fold cross-validation, provides a more accurate estimate of test accuracy compared to a single data split. However, it's important to clarify that the validation accuracy obtained through cross-validation is not the same as the test accuracy; it's an estimate of how well your model is expected to perform on unseen data. Here's why it's considered more accurate:

Better Representation of Data: Cross-validation divides your dataset into multiple subsets and ensures that each subset serves as both a training and validation set. This means your model is evaluated on different portions of the data, providing a more comprehensive view of its performance across various data points and scenarios.

Reduced Bias: With a single data split, you might be lucky or unlucky in terms of which data points end up in the validation set. Cross-validation mitigates this bias by averaging performance over multiple splits, making the estimate less sensitive to the particular choice of data.

3)The number of iterations, or folds, in cross-validation can have an effect on the estimate of your model's performance. Here's how it works:

Effect of Increasing Iterations:

As you increase the number of iterations (k) in k-fold cross-validation, you typically obtain a more robust and reliable estimate of your model's performance.
With a higher number of iterations, your model is trained and validated on a larger number of different data splits, which provides a better overall assessment of how well it generalizes.
The estimate of your model's performance becomes less sensitive to the specific random selection of data points in each fold, reducing variability in the results.
Effect of Decreasing Iterations:

If you decrease the number of iterations, you might obtain a less reliable estimate of your model's performance.
With fewer iterations, your estimate can be influenced by the randomness of data splits to a greater degree, potentially leading to less consistent results.
There's a risk of underestimating or overestimating your model's true generalization performance.
However, it's important to strike a balance. Increasing the number of iterations requires more computational resources and time, and there may be diminishing returns. Common choices for k include 5, 10, or even 20, depending on the size of your dataset and the computational resources available.

In summary, increasing the number of iterations in cross-validation can provide a better estimate of your model's generalization performance by reducing the impact of random data splits. However, there's no need to use an extremely high number of iterations; you should choose a value that balances the need for accuracy with practicality and resource constraints.





Send a message


Free Research Preview. ChatGPT

4)Mitigating Small Dataset Issues:

When you have a very small training dataset, increasing the number of iterations in cross-validation (e.g., using a higher k value) can allow your model to train and validate on different subsets of the data more frequently.
This can help provide a more comprehensive assessment of your model's generalization performance and reduce the impact of data randomness.
Limitations of Small Datasets:

Small datasets inherently have limited information for your model to learn from. No amount of cross-validation iterations can create more data or overcome the fundamental limitations of a small dataset.
Small datasets are more prone to overfitting, where a model learns to perform well on the limited training data but struggles to generalize to new, unseen data.
Resource Considerations:

Increasing the number of iterations also increases computational demands and time requirements. It might not always be feasible or efficient to use a very high k value.
In practice, while increasing iterations can help, it's often advisable to work with larger training datasets whenever possible to improve your model's ability to learn and generalize. If dealing with a small dataset is unavoidable, consider techniques like data augmentation (for image data), transfer learning (leveraging pre-trained models), or feature engineering to make the most of the available